In [1]:
%%configure -f
{
  "conf": {
    "spark.pyspark.python": "python3"
  }
}


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1763344109923_0007,pyspark,idle,Link,Link,None,
10,application_1763344109923_0013,pyspark,idle,Link,Link,None,


In [2]:
spark
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
12,application_1763344109923_0015,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-12>

In [11]:
df = spark.read.option("header", True)\
               .option("inferSchema", True)\
               .option("sep", ",")\
               .csv("s3://emrsamuel/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df.show(5)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+---------

In [13]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [14]:
df = df.toDF(*[c.strip().lower().replace(" ", "_").replace("ó","o").replace("á","a").replace("í","i").replace("é","e").replace("ú","u") for c in df.columns])
df.show(5)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha_reporte_web|id_de_caso|fecha_de_notificación|código_divipola_departamento|nombre_departamento|código_divipola_municipio|nombre_municipio|edad|unidad_de_medida_de_edad|sexo|tipo_de_contagio|ubicación_del_caso|estado|código_iso_del_país|nombre_del_país|recuperado|fecha_de_inicio_de_síntomas|fecha_de_muerte|fecha_de_diagnóstico|fecha_de_recuperación|tipo_de_recuperación|pertenencia_étnica|nombre_del_grupo_étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+---------

In [15]:
from pyspark.sql.functions import col, when

df = df.withColumn(
    "edad_grupo",
    when(col("edad") < 18, "menor_18")
    .when((col("edad") >= 18) & (col("edad") <= 40), "18-40")
    .when((col("edad") > 40) & (col("edad") <= 60), "41-60")
    .otherwise("mayor_60")
)

df.select("edad", "edad_grupo").show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+
|edad|edad_grupo|
+----+----------+
|  19|     18-40|
|  34|     18-40|
|  50|     41-60|
|  55|     41-60|
|  25|     18-40|
|  27|     18-40|
|  85|  mayor_60|
|  22|     18-40|
|  28|     18-40|
|  36|     18-40|
+----+----------+
only showing top 10 rows

In [16]:
df_activos = df.filter(col("estado") == "Leve")  # o el valor que indique activo
df_activos.show(5)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+----------+
|fecha_reporte_web|id_de_caso|fecha_de_notificación|código_divipola_departamento|nombre_departamento|código_divipola_municipio|nombre_municipio|edad|unidad_de_medida_de_edad|sexo|tipo_de_contagio|ubicación_del_caso|estado|código_iso_del_país|nombre_del_país|recuperado|fecha_de_inicio_de_síntomas|fecha_de_muerte|fecha_de_diagnóstico|fecha_de_recuperación|tipo_de_recuperación|pertenencia_étnica|nombre_del_grupo_étnico|edad_grupo|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+---------

In [17]:
df_departamento = df.groupBy("nombre_departamento")\
                    .count()\
                    .orderBy("count", ascending=False)
df_departamento.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|nombre_departamento|count|
+-------------------+-----+
|             BOGOTA|  401|
|              VALLE|  148|
|          ANTIOQUIA|  106|
|       CUNDINAMARCA|   49|
|          CARTAGENA|   39|
|          RISARALDA|   34|
|       BARRANQUILLA|   31|
|              HUILA|   30|
|            QUINDIO|   23|
|    NORTE SANTANDER|   19|
+-------------------+-----+
only showing top 10 rows

In [18]:
df_ciudad = df.groupBy("nombre_municipio")\
              .count()\
              .orderBy("count", ascending=False)
df_ciudad.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|nombre_municipio|count|
+----------------+-----+
|          BOGOTA|  401|
|            CALI|  101|
|        MEDELLIN|   63|
|       CARTAGENA|   39|
|    BARRANQUILLA|   31|
|           NEIVA|   27|
|         PEREIRA|   25|
|         PALMIRA|   22|
|      VALLEDUPAR|   16|
|          CUCUTA|   15|
+----------------+-----+
only showing top 10 rows

In [19]:
df_fecha = df.groupBy("fecha_reporte_web")\
             .count()\
             .orderBy("count", ascending=False)
df_fecha.show(10)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+
|fecha_reporte_web|count|
+-----------------+-----+
|31/3/2020 0:00:00|  107|
|24/3/2020 0:00:00|  105|
|30/3/2020 0:00:00|   96|
| 1/4/2020 0:00:00|   95|
|29/3/2020 0:00:00|   94|
|23/3/2020 0:00:00|   73|
|28/3/2020 0:00:00|   69|
|25/3/2020 0:00:00|   61|
|27/3/2020 0:00:00|   49|
|20/3/2020 0:00:00|   48|
+-----------------+-----+
only showing top 10 rows

In [20]:
df_edad = df.groupBy("edad_grupo")\
            .count()\
            .orderBy("edad_grupo")
df_edad.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|edad_grupo|count|
+----------+-----+
|     18-40|  476|
|     41-60|  332|
|  mayor_60|  159|
|  menor_18|   33|
+----------+-----+

In [21]:
df.createOrReplaceTempView("covid")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
spark.sql("""
SELECT nombre_departamento, COUNT(*) AS casos
FROM covid
GROUP BY nombre_departamento
ORDER BY casos DESC
LIMIT 10
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----+
|nombre_departamento|casos|
+-------------------+-----+
|             BOGOTA|  401|
|              VALLE|  148|
|          ANTIOQUIA|  106|
|       CUNDINAMARCA|   49|
|          CARTAGENA|   39|
|          RISARALDA|   34|
|       BARRANQUILLA|   31|
|              HUILA|   30|
|            QUINDIO|   23|
|    NORTE SANTANDER|   19|
+-------------------+-----+

In [23]:
spark.sql("""
SELECT nombre_municipio, COUNT(*) AS casos
FROM covid
GROUP BY nombre_municipio
ORDER BY casos DESC
LIMIT 10
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----+
|nombre_municipio|casos|
+----------------+-----+
|          BOGOTA|  401|
|            CALI|  101|
|        MEDELLIN|   63|
|       CARTAGENA|   39|
|    BARRANQUILLA|   31|
|           NEIVA|   27|
|         PEREIRA|   25|
|         PALMIRA|   22|
|      VALLEDUPAR|   16|
|         ARMENIA|   15|
+----------------+-----+

In [24]:
spark.sql("""
SELECT fecha_reporte_web, COUNT(*) AS casos
FROM covid
GROUP BY fecha_reporte_web
ORDER BY casos DESC
LIMIT 10
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----+
|fecha_reporte_web|casos|
+-----------------+-----+
|31/3/2020 0:00:00|  107|
|24/3/2020 0:00:00|  105|
|30/3/2020 0:00:00|   96|
| 1/4/2020 0:00:00|   95|
|29/3/2020 0:00:00|   94|
|23/3/2020 0:00:00|   73|
|28/3/2020 0:00:00|   69|
|25/3/2020 0:00:00|   61|
|27/3/2020 0:00:00|   49|
|20/3/2020 0:00:00|   48|
+-----------------+-----+

In [25]:
spark.sql("""
SELECT
    CASE
        WHEN edad < 18 THEN 'menor_18'
        WHEN edad BETWEEN 18 AND 40 THEN '18-40'
        WHEN edad BETWEEN 41 AND 60 THEN '41-60'
        ELSE 'mayor_60'
    END AS edad_grupo,
    COUNT(*) AS casos
FROM covid
GROUP BY
    CASE
        WHEN edad < 18 THEN 'menor_18'
        WHEN edad BETWEEN 18 AND 40 THEN '18-40'
        WHEN edad BETWEEN 41 AND 60 THEN '41-60'
        ELSE 'mayor_60'
    END
ORDER BY edad_grupo
""").show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|edad_grupo|casos|
+----------+-----+
|     18-40|  476|
|     41-60|  332|
|  mayor_60|  159|
|  menor_18|   33|
+----------+-----+

In [26]:
# CSV
df_departamento.coalesce(1).write.csv("s3://emrsamuel/resultados/departamentos.csv", header=True)
df_ciudad.coalesce(1).write.csv("s3://emrsamuel/resultados/ciudades.csv", header=True)
df_fecha.coalesce(1).write.csv("s3://emrsamuel/resultados/fechas.csv", header=True)
df_edad.coalesce(1).write.csv("s3://emrsamuel/resultados/edades.csv", header=True)

# Parquet
df_departamento.write.parquet("s3://emrsamuel/resultados/departamentos.parquet")
df_ciudad.write.parquet("s3://emrsamuel/resultados/ciudades.parquet")
df_fecha.write.parquet("s3://emrsamuel/resultados/fechas.parquet")
df_edad.write.parquet("s3://emrsamuel/resultados/edades.parquet")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…